In [100]:
import numpy as np
import os
import pandas as pd
from collections import Counter
import psutil

In [104]:
psutil.Process(os.getpid()).memory_info().rss / 1024 / 1024

636.55859375

In [4]:
os.chdir('..')
os.getcwd()

'/home/ec2-user/SageMaker/AmazonSageMaker-bluewater-nlp'

In [67]:
emb_fp = '../pretrained/rebuild.vectors.npy'
vocab_fp = '../pretrained/rebuild.vocab.txt'
dataset_vocab_fp = '../wrk/restroom-dataset.v2.vocab.csv'

In [6]:
emb_arr = np.load(emb_fp)
emb_arr.shape

(196622, 300)

In [35]:
with open(vocab_fp) as inp:
    vocab_arr = [l.rstrip() for l in inp]
vocab_arr = np.array(vocab_arr)
vocab_arr.shape

(196622,)

In [95]:
def get_pos(w):
    i = w.rfind('_')
    return w[i+1:] if i > 0 else None

In [99]:
vocab_pos_freq_ss = pd.Series(Counter(get_pos(w) for w in vocab_arr)).sort_values(ascending=False)
vocab_pos_freq_ss

NOUN     86861
ADJ      31561
X        31544
PROPN    25945
VERB     16321
ADV       4194
INTJ       137
NUM         57
NaN          2
dtype: int64

In [105]:
vocab_pos_freq_ss / vocab_pos_freq_ss.sum()

NOUN     0.441766
ADJ      0.160516
X        0.160430
PROPN    0.131954
VERB     0.083007
ADV      0.021330
INTJ     0.000697
NUM      0.000290
NaN      0.000010
dtype: float64

# Define a function to get the most similar words

In [36]:
vocab_idx = {
    w : i
    for i, w in enumerate(vocab_arr)
}

In [58]:
def get_closest_words(w, n_top=15):
    w_i = vocab_idx.get(w, -1)
    if w_i < 0:
        # TODO list suggestions using argmin over edit distance
        raise ValueError("Can't find word '%s' in the vocab" % w)
    w_emb = emb_arr[w_i]
    dist_arr = np.sqrt(np.power(emb_arr - w_emb, 2).sum(-1))
    top_indices = np.argpartition(dist_arr, n_top)[:n_top]
    top_distances = dist_arr[top_indices]
    return pd.Series(top_distances, index=vocab_arr[top_indices]).sort_values()

In [124]:
get_closest_words('хуй_NOUN')

хуй_NOUN        0.000000
хер_NOUN        0.518857
пизда_NOUN      0.656288
жопа_NOUN       0.658848
залупа_NOUN     0.701741
ебать_VERB      0.710420
срака_NOUN      0.724436
писюн_NOUN      0.736951
писька_NOUN     0.740310
блять_VERB      0.749357
блядь_NOUN      0.752477
бля_INTJ        0.758907
дрочить_VERB    0.762496
задница_NOUN    0.776444
ебаться_VERB    0.782574
dtype: float32

# Investigate OOV in the dataset

In [91]:
dataset_vocab_ss = pd.read_csv(dataset_vocab_fp, header=None, squeeze=True, index_col=0)
dataset_vocab_ss.index.name = 'Token'
dataset_vocab_ss.name = 'Frequency'
dataset_vocab_ss.size

181

In [92]:
dataset_vocab_ss[~dataset_vocab_ss.index.isin(vocab_idx)]

Token
я_PRON                   21
в_ADP                    15
не_PART                   7
опять_ADV                 7
сейчас_ADV                5
кто_PRON                  5
на_ADP                    4
мой_DET                   3
такой_DET                 3
какой_DET                 3
как_SCONJ                 3
из_ADP                    3
ну_PART                   3
никогда_ADV               2
он_PRON                   2
как_ADV                   2
и_SCONJ                   2
за_ADP                    2
все_PRON                  2
вот_PART                  2
быть_VERB                 2
уже_ADV                   2
то_PART                   2
только_PART               2
у_ADP                     1
еще_ADV                   1
ебаша_NOUN                1
тут_ADV                   1
ты_PRON                   1
дыка_NOUN                 1
тот_DET                   1
когда_SCONJ               1
футбок_NOUN               1
что_SCONJ                 1
врота_NOUN                1
что_PRON      